#  Materiały do zadań - wydźwięk i tematy

## Analiza wydźwięku

Analiza wydźwięku (ang. _sentiment analysis_) polega na określeniu nacechowania emocjonalnego tekstu. 

Jest to rozbudowany i mocno subiektywny problem - tekst ma inny wydźwięk dla autora tekstu, inny dla osoby czytającej, inny dla "podmiotu lirycznego"; mocno zależy też od kontekstu kulturowego.

Najczęstszym przypadkiem jest uproszczenie zadania do trójklasowej klasyfikacji - podziału na teksty `pozytywne`, `neutralne`, `negatywne` - lub regresji z zadanej skali - np. `[0; 1]`

Istnieje wiele gotowych do użycia rozwiązań pozwalających analizować sentyment:
- [sentimentPL](https://pypi.org/project/sentimentpl/) - dla PL
- [Vader](https://pypi.org/project/vaderSentiment/) - dla EN

Alternatywnie, dzięki dużej ilości dostępnych w sieci zbiorów danych i pretrenowanych modeli językowych, wyuczenie całkiem dobrej jakości modelu samodzielnie nie powinno nastręczać dużych trudności.

## Modelowanie tematyczne

Jest to metoda statystycznego klastrowania dokumentów - a nawet bardziej słów - w grupy podobne tematycznie. 

Jedną z popularniejszych metod jest [LDA - Latend Dirichlet Allocation](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation). 

Jej podstawowym założeniem jest:
``` 
Każdy tekst może być przedstawiony jako rozkład tematów.
Każdy temat może być przedstawiony jako rozkład słów.
```

[Więcej informacji i przykładów](https://medium.com/@lettier/how-does-lda-work-ill-explain-using-emoji-108abf40fa7d)

### LDA w Gensim

Użyjemy bibliteki Gensim do przeprowadzenia przykładowego modelowania tematycznego

```
# instalacja potrzebnych bibliotek

pip install spacy
python -m spacy download en_core_web_sm
pip install gensim
pip install pyLDAvis
pip install sklearn
```

Dla celów prezentacji, użyjemy popularnego zbioru danych [20newsgroups](http://qwone.com/~jason/20Newsgroups/), dostępnego poprzez interface _sklearn.datasets_

In [1]:
from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups()

data = newsgroups["data"][:25000]
print(data[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







Dataset zawiera maile wysyłane w ramach grup dyskusyjnych - jest więc dość mocno zaszumiony. By uzyskać interesującą nas semantykę, dokonamy podstawowych kroków preprocessingu przy użyciu frameworku Spacy

In [2]:
import spacy

# wczytujemy moduł językowy spacy - obsługa innych języków: https://spacy.io/usage/models
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

cleaned = []

for text in data:
    doc = nlp(text)

    cleaned.append(
        [
            token.lemma_.lower()
            for token in doc
            if not (
                token.is_stop
                or token.is_punct
                or token.like_email
                or token.like_url
                or token.like_num
                or token.is_digit
                or token.pos_ not in ["NOUN", "ADJ", "VERB", "ADV"]
            )
        ]
    )

print(cleaned[0])

['thing', 'subject', 'car', 'host', 'line', 'wonder', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', '70', 'call', 'door', 'small', 'addition', 'bumper', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'engine', 'spec', 'year', 'production', 'car', 'history', 'info', 'funky', 'look', 'car', 'e', 'mail', 'thank', 'bring', 'neighborhood']


In [3]:
from gensim.corpora import Dictionary

# tworzymy słownik na bazie słów z datasetu
id2word = Dictionary(cleaned)

# filtrujemy zbyt rzadkie i zbyt popularne słowa
id2word.filter_extremes(no_below=20, no_above=0.5)

# wektoryzujemy słowa przy pomocy bag-of-words
corpus = [id2word.doc2bow(text) for text in cleaned]

print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 5), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1)]


In [4]:
from gensim.models.ldamulticore import LdaMulticore

# dokonujemy modelowania tematycznego
lda_model = LdaMulticore(corpus=corpus, id2word=id2word, num_topics=5)

In [5]:
# wyświetlenie definicji tematów
lda_model.print_topics()

[(0,
  '0.010*"|" + 0.008*"know" + 0.007*"people" + 0.006*"article" + 0.006*"x" + 0.006*"time" + 0.006*"say" + 0.005*"look" + 0.005*"year" + 0.005*"way"'),
 (1,
  '0.007*"people" + 0.007*"article" + 0.005*"need" + 0.005*"problem" + 0.005*"work" + 0.005*"good" + 0.005*"thing" + 0.005*"think" + 0.004*"know" + 0.004*"system"'),
 (2,
  '0.027*"|" + 0.010*"+" + 0.009*"article" + 0.008*"think" + 0.008*"know" + 0.008*"people" + 0.007*"time" + 0.006*"good" + 0.006*"work" + 0.006*"use"'),
 (3,
  '0.012*"|" + 0.008*"article" + 0.008*"think" + 0.007*"know" + 0.007*"use" + 0.006*"people" + 0.006*"say" + 0.005*"host" + 0.005*"go" + 0.005*"time"'),
 (4,
  '0.030*"|" + 0.007*"article" + 0.007*"know" + 0.005*"right" + 0.005*"think" + 0.005*"good" + 0.005*"year" + 0.005*"use" + 0.005*"say" + 0.004*"people"')]

In [6]:
# mieszanka tematów dokumentu 1
lda_model[corpus[0]]

[(0, 0.97892857)]

#### Mierzenie jakości wyników modelowania

Do sprawdzenia jakości zbadanych tematów użyjemy dwóch metryk - perplexity i coherence

*Perplexity* określa zdolność modelu probabilistycznego do poprawnego modelowania próbki. Im wyższe perplexity, tym model jest mniej pewny swoich wskazań, stąd **im niższe perplexity, tym lepiej**

*Coherence* mierzy spójność semantyczną tematu. **Im wyższe coherence, tym lepiej**

[Więcej informacji o mierzeniu jakości modelowania tematycznego](https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0)

In [7]:
from gensim.models import CoherenceModel

print("\nPerplexity: ", lda_model.log_perplexity(corpus))

coherence_model_lda = CoherenceModel(
    model=lda_model, texts=cleaned, dictionary=id2word, coherence="c_v"
)
coherence_lda = coherence_model_lda.get_coherence()
print("\nCoherence Score: ", coherence_lda)


Perplexity:  -7.524670794746033

Coherence Score:  0.32994280458668285


### Ręczna analiza i wizualizacja wyników analizy

In [8]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.000708 -0.006958       1        1  24.737140
4     -0.006443 -0.005997       2        1  19.874502
2     -0.009885  0.011692       3        1  18.898713
1      0.014569  0.005958       4        1  18.743476
3      0.001051 -0.004696       5        1  17.746169, topic_info=      Term          Freq         Total Category  logprob  loglift
790      |  14841.000000  14841.000000  Default  30.0000  30.0000
666      +   2103.000000   2103.000000  Default  29.0000  29.0000
832      x   3419.000000   3419.000000  Default  28.0000  28.0000
293  think   5462.000000   5462.000000  Default  27.0000  27.0000
40    card   1374.000000   1374.000000  Default  26.0000  26.0000
..     ...           ...           ...      ...      ...      ...
295    try    484.305190   2755.801722   Topic5  -5.8170  -0.0097
17    look    517.137853   3319.993816   Topic5  -5.7514  -0.1304
373   work    540.403118   3804.366786   Topic5  -5.7074  -0.2226
29   thing    510.727666   3417.924803   Topic5  -5.7638  -0.1720
31    year    470.318135   3902.687229   Topic5  -5.8463  -0.3870

[425 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
666       1  0.076050         +
666       2  0.026618         +
666       3  0.804230         +
666       4  0.059889         +
666       5  0.033272         +
...     ...       ...       ...
2849      1  0.047242  |_____|/
2849      2  0.708627  |_____|/
2849      3  0.141725  |_____|/
2849      4  0.047242  |_____|/
2849      5  0.094484  |_____|/

[1410 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 3, 2, 4])

### Jak wyznaczyć odpowiednią ilość tematów?

LDA jest parametryzowane ilością tematów. Wartość tego parametru ściśle zależy od analizowanych danych.

Jest jednakże kilka popularnych wskazówek które moga być pomocne przy wyborze ilości tematów:
- zbudować kilka modeli z różnymi wartościami parametru i wybrac ten o najlepszych wartościach perplexity i coherence
- przejrzeć ręcznie słowa zawarte w tematach - jeśli takie same słowa powtarzają się w wielu tematach, prawdopodobnie ilośc tematów jest zbyt duża
- w analizie pyLDAvis, dobry model tematyczny powinien generowac duże klastry w każdej ćwiartce wykresu. Małe i przecinające się klastry sugerują zbyt dużą ilość tematów